In [1]:
import pandas as pd
import glob
import os

In [ ]:
filename = "Data/GSM3533230_CGND-HRA-00013_counts.txt"

data = pd.read_csv(filename, sep="\t", names=[filename], skiprows=1)
data.T.shape

In [28]:
path = "Data"
filenames = glob.glob(path + "/*.txt")

dfs = []
for filename in filenames:
    data = pd.read_csv(filename, sep="\t", names=[filename], skiprows=1)
    df_t = data.T
    dfs.append(df_t)

# Concatenate all data into one DataFrame
big_frame = pd.concat(dfs)
big_frame.head()

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A3GALT2,A4GALT,A4GNT,...,X6B_LINE:CR1:LINE,X7A_LINE:CR1:LINE,X7B_LINE:CR1:LINE,X7C_LINE:CR1:LINE,X7D_LINE:CR1:LINE,X8_LINE:CR1:LINE,X9_LINE:L1:LINE,Zaphod2:hAT-Tip100:DNA,Zaphod3:hAT-Tip100:DNA,Zaphod:hAT-Tip100:DNA
Data/GSM3533382_CGND-HRA-00756_counts.txt,42,189,0,4052,92,201,1,8,76,1,...,1023,2363,2137,1265,100,607,140,551,2269,2233
Data/GSM3533252_CGND-HRA-00064_counts.txt,37,110,0,4660,102,64,7,1,68,0,...,880,2077,1908,1072,119,478,129,525,1821,1921
Data/GSM3533337_CGND-HRA-00421_counts.txt,31,110,0,1484,84,60,1,1,12,0,...,656,1357,1345,749,61,341,102,346,1122,1437
Data/GSM3533314_CGND-HRA-00358_counts.txt,14,41,0,2185,105,198,4,0,24,0,...,958,2017,1817,1087,99,527,145,446,1522,1757
Data/GSM3533391_CGND-HRA-01219_counts.txt,37,118,0,1245,104,168,1,1,21,1,...,847,1822,1625,903,96,472,120,415,1556,1768


In [ ]:
df = pd.read_csv('Data/GSM3533230_CGND-HRA-00013_counts.txt', sep="\t",skiprows=1,names=["GSM3533230_CGND-HRA-00013_counts"])
df_t = df.T
df_t.columns = df_t.iloc[0]
df_t.head()

